In [1]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0430 18:52:31.953288 139855783298816 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [4]:
!mkdir bert_delete

In [5]:
OUTPUT_DIR = "bert_delete"

In [6]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'bert_delete'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: bert_delete *****


#Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [7]:
from tensorflow import keras
import os
import re

In [8]:
Conventional = ["Generic correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Choice of tense", "Tense form", "Voice", "Modals", "Verb pattern", "Intransitive verb", "Transitive verb", "Reflexive verb", "Verb with as", "Ambitransitive verb", "Two verbal forms in the predicate", "Verb + Infinitive", "Verb + Gerund", "Verb + Infinitive OR Gerund", "Verb + Bare Infinitive", "Verb + Object/Addressee + Bare Infinitive", "Infinitive Restoration Alternation", "Verb + Participle", "Get + participle", "Complex-object verb", "Verbal idiom", "Prepositional or phrasal verb", "Dative verb with alternation", "Verb followed by a clause", "Verb + that/WH + Clause", "Verb + if/whether + clause", "Verb + that + Subjunctive clause", "Verb + it + Conj + Clause", "Participial construction", "Infinitive construction", "Gerund phrase", "Nouns", "Countable/uncountable", "Prepositional noun", "Possessive form of a noun", "Noun as an attribute", "Noun + Infinitive", "Noun number", "Prepositions", "Conjunctions", "Adjectives", "Comparative degree of adjectives", "Superlative degree of adjectives", "Prepositional adjective", "Adjective as a collective noun", "Adverbs", "Comparative degree of adverbs", "Superlative degree of adverbs", "Prepositional adverb", "Numerals", "Pronouns", "Agreement", "Word order", "Standard word order", "Emphatic shift", "Cleft sentence", "Interrogative word order", "Incomplete sentence", "Exclamation", "Title structure", "Note structure", "Conditionals", "Attributes", "Relative clause", "Defining relative clause", "Non-defining relative clause", "Coordinate relative clause", "Attributive participial construction", "Parallel constructions", "Negation", "Comparative construction", "Numerical comparison", "Confusion of structures", "Vocabulary", "Word choice", "Choice of lexical item", "Words often confused", "Choice of a part of lexical item", "Absence of certain components of a collocation", "Redundant word(s)", "Word formation", "Derivational affixes", "Formational suffix", "Formational prefix", "Confusion of categories", "Compound word", "Discourse", "Referential device", "Coherence", "Linking device", "Inappropriate register", "Absence of a component in clause or sentence", "Redundant component in clause or sentence", "Absence of necessary explanation or detail", "Deletion"]
Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]
translate_dict = {e[0]: e[1] for e in zip(Conventional, Tags)}

error_type = "Deletion" #@param ["Spelling", "Choice of lexical item", "Deletion", "Prepositions", "Agreement", "Noun number", "Confusion of categories", "Referential device", "Capitalisation", "Words often confused"]
error_ratio = "ToFifteen" #@param ["ToFifteen", "AugmentedRatio"]

error_type = translate_dict[error_type]

It's about time we named our model

In [9]:
modelname = "delete_ToFifteen" # @param {type:"string"}

In [10]:
import shutil

filename = error_ratio+"/train/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'.')
trn = pd.read_json(error_type+".json").reset_index(drop=True)
trn["is_error"] = trn["is_error"].astype(int)

filename = error_ratio+"/test/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'./'+error_type+'_test.json')
tst = pd.read_json(error_type+"_test.json").reset_index(drop=True)
tst["is_error"] = tst["is_error"].astype(int)

In [11]:
len(trn)

66022

To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [12]:
train = trn
test = tst

In [13]:
train.columns

Index(['context', 'is_error', 'path', 'substring'], dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [14]:
DATA_COLUMN = 'context'
SUBSTR_COLUMN = 'substring'
LABEL_COLUMN = 'is_error'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [15]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [16]:
train_InputExamples

0        <bert.run_classifier.InputExample object at 0x...
1        <bert.run_classifier.InputExample object at 0x...
2        <bert.run_classifier.InputExample object at 0x...
3        <bert.run_classifier.InputExample object at 0x...
4        <bert.run_classifier.InputExample object at 0x...
5        <bert.run_classifier.InputExample object at 0x...
6        <bert.run_classifier.InputExample object at 0x...
7        <bert.run_classifier.InputExample object at 0x...
8        <bert.run_classifier.InputExample object at 0x...
9        <bert.run_classifier.InputExample object at 0x...
10       <bert.run_classifier.InputExample object at 0x...
11       <bert.run_classifier.InputExample object at 0x...
12       <bert.run_classifier.InputExample object at 0x...
13       <bert.run_classifier.InputExample object at 0x...
14       <bert.run_classifier.InputExample object at 0x...
15       <bert.run_classifier.InputExample object at 0x...
16       <bert.run_classifier.InputExample object at 0x.

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [17]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"

class NonMaskOmittingTokenizer(bert.tokenization.FullTokenizer):
  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    for index, item in enumerate(split_tokens):
      if index >= len(split_tokens)-2:
        break
      if item == '[' and split_tokens[index + 1] == 'MA' and split_tokens[index + 2] == '##S' and split_tokens[index + 3] == '##K' and split_tokens[index + 4] == ']':
        split_tokens[index] = "[MASK]"
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]

    return split_tokens

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return NonMaskOmittingTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 18:53:02.402678 139855783298816 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [18]:
tokenizer.tokenize("This here's an example of using the BERT [MASK] tokenizer")

['This',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'B',
 '##ER',
 '##T',
 '[MASK]',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [19]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 66022


I0430 18:53:23.833483 139855783298816 tf_logging.py:115] Writing example 0 of 66022


INFO:tensorflow:*** Example ***


I0430 18:53:23.836577 139855783298816 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0430 18:54:27.668916 139855783298816 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


I0430 18:54:27.670088 139855783298816 tf_logging.py:115] label: 1 (id = 1)


INFO:tensorflow:Writing example 10000 of 16506


I0430 18:54:37.461646 139855783298816 tf_logging.py:115] Writing example 10000 of 16506


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [20]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [21]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [22]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [23]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS) * 2
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print(num_train_steps)

12378


In [24]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [25]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_evaluation_master': '', '_model_dir': 'bert_delete', '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_save_checkpoints_steps': 500, '_is_chief': True, '_protocol': None, '_tf_random_seed': None, '_master': '', '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_eval_distribute': None, '_save_summary_steps': 100, '_train_distribute': None, '_experimental_distribute': None, '_device_fn': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f321b35ed30>, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_global_id_in_cluster': 0}


I0430 18:54:44.567561 139855783298816 tf_logging.py:115] Using config: {'_evaluation_master': '', '_model_dir': 'bert_delete', '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_save_checkpoints_steps': 500, '_is_chief': True, '_protocol': None, '_tf_random_seed': None, '_master': '', '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_eval_distribute': None, '_save_summary_steps': 100, '_train_distribute': None, '_experimental_distribute': None, '_device_fn': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f321b35ed30>, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_global_id_in_cluster': 0}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [26]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [27]:
print('Beginning Training!')
current_time = datetime.now()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    with tf.device('/gpu:0'):
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0430 18:55:14.153069 139855783298816 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 18:55:17.504966 139855783298816 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 12378 into bert_delete/model.ckpt.


I0430 20:15:16.944675 139855783298816 tf_logging.py:115] Saving checkpoints for 12378 into bert_delete/model.ckpt.


INFO:tensorflow:Loss for final step: 0.0003956622.


I0430 20:15:20.760783 139855783298816 tf_logging.py:115] Loss for final step: 0.0003956622.


Training took time  1:20:36.128078


Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

Now let's use our test data to see how well our model did:

In [28]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [38]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0430 21:55:08.806004 139855783298816 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 21:55:10.980270 139855783298816 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 12378: auc = 0.5747086, eval_accuracy = 0.94359624, f1_score = 0.25340813, false_negatives = 874.0, false_positives = 57.0, global_step = 12378, loss = 0.46494824, precision = 0.7348837, recall = 0.15310077, true_negatives = 15417.0, true_positives = 158.0


I0430 21:56:27.166480 139855783298816 tf_logging.py:115] Saving dict for global step 12378: auc = 0.5747086, eval_accuracy = 0.94359624, f1_score = 0.25340813, false_negatives = 874.0, false_positives = 57.0, global_step = 12378, loss = 0.46494824, precision = 0.7348837, recall = 0.15310077, true_negatives = 15417.0, true_positives = 158.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 12378: bert_delete/model.ckpt-12378


I0430 21:56:27.168566 139855783298816 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 12378: bert_delete/model.ckpt-12378


{'auc': 0.5747086,
 'eval_accuracy': 0.94359624,
 'f1_score': 0.25340813,
 'false_negatives': 874.0,
 'false_positives': 57.0,
 'global_step': 12378,
 'loss': 0.46494824,
 'precision': 0.7348837,
 'recall': 0.15310077,
 'true_negatives': 15417.0,
 'true_positives': 158.0}

#Extracting the trained model


Now let's save our model:

In [0]:
# Export the model
def serving_input_fn():
  with tf.variable_scope("foo"):
    feature_spec = {
        "input_ids": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "input_mask": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "segment_ids": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "label_ids": tf.FixedLenFeature([], tf.int64),
      }
    serialized_tf_example = tf.placeholder(dtype=tf.string,
                                           shape=[None],
                                           name='input_example_tensor')
    receiver_tensors = {'examples': serialized_tf_example}
    features = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

EXPORT_DIR = './Exported models/'+modelname
estimator._export_to_tpu = False  # this is important
path = estimator.export_savedmodel(EXPORT_DIR, serving_input_fn)
print(path)

Check if we can load it correctly:

In [0]:
subdirs = [x for x in Path(EXPORT_DIR).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

In [0]:
from tensorflow.contrib import predictor

predict_fn = predictor.from_saved_model(latest)

In [53]:
def getPrediction(in_sentences):
  labels = ["Not an error", "Is an error"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [44]:
import nltk
import re
import itertools

nltk.download('perluniprops')
nltk.download('punkt')

[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [54]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

def annotate(text):
  current_time = datetime.now()
  raw = text
  raw = re.sub(r'\s', ' ', raw)
  raw = re.sub(r'( )+', ' ', raw)
  sentences = nltk.sent_tokenize(raw)
  wordsoup = []
  annotation_set = []
  checking_ids = []
  tokens = []
  J = 0
  for sentence in sentences:
    wordsoup.append(tknzr.tokenize(sentence))
  for i in range(len(wordsoup)):
    tokenized_sentence = wordsoup[i]
    for j in range(len(tokenized_sentence)):
      token = tokenized_sentence[j]
      if not re.search(r'[a-zA-Z]', token):
        tokens.append([token, 0])
      else:
        tokens.append([token, None])
        substring = token
        si = i-2
        if si < 0:
          si = 0
        detokenizing_string = wordsoup[si:i] + [tokenized_sentence[:j] + ['[MASK]'] + tokenized_sentence[j+1:]] + wordsoup[i+1:i+3]
        detokenizing_string = [item for sublist in detokenizing_string for item in sublist]
        entry = detokenizer.detokenize(detokenizing_string, return_str=True)
        annotation_set.append([entry, substring])
        checking_ids.append(J)
      J += 1
  predictions = getPrediction(annotation_set[:128])
  for p in range(len(predictions)):
    if predictions[p][-1] == 'Is an error':
      tokens[checking_ids[p]][1] = 1
    else:
      tokens[checking_ids[p]][1] = 0
  print('\n')
  print("Annotation took time ", datetime.now() - current_time)
  return tokens

def print_annotated_webpage(tokens, title):
  out = '<html>\n<head>\n<title>'+title+'</title>\n<meta charset="utf-8">\n<style type="text/css">\n.red {\n\tbackground: #ffdddd;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>'
  outsoup = []
  for token in tokens:
    if token[1] == 1:
      token[0] = '<div class="red">'+token[0]+'</div>'
    outsoup.append(token[0])
  bodystring = detokenizer.detokenize(outsoup, return_str=True)
  out += bodystring
  out += '</body>\n</html>'
  return out

In [55]:
import json

curname = "wsites_" + modelname + ".json"

test_texts = {"./data/exam/exam2017/VSa_75_1.txt": "The graph provide us information about print and eBook market in USA, Germany, and UK in 2014 annd 2018. The biggest number in this tip of print and eBook in USA. HowEver, number of eBook and print market also big in Germany. In China we can see less number of market and in the UK this number the smallest between all country in 2014 and 2018. In USA's graph we see that number of eBook market increas in 2018 on 3,5% comparasion with 2014 and number of print market desceased on 3% in 2018. In Germany number of print market stay the same in 2018 as in 2014 but number of eBook market small increas in 2018. This number is 0,5%. In China number of print fall in 2018 on 0,6% but number of E book increase on 0,5%. In UK we can see the same tend. Number of prin markets fall in 2018 on 0,6%. Also number of Ebook market rise on 0,5%. Finally we can see that in all four countries number of eBook market rise in 2018 and number of print market will fall in 2018. However Germany is stay at the some number of print market.\n", "./data/2012-2014/esl_00637.txt": "It is believed that students in schools should be selected according to their academic abilities, but others don't agree with them, because they concerned that students with different abilities have to study together.\nTo begin with, i think that the both facts has their advantages and disadvantages. Firstly, i want to discuss schools with students with the same level of abilities. On the one hand, i'm concerned that if some schools selected their students with a high level of skills, they just want to have spotless reputation as an excellent school. On the other hand, to my mind, if some school had students with a low level of abilities, that may tell as, that they are some kind of school for children, who are unable to study good, for instance for disabled children. I am also concerned that it is correct, because it will help to avoid  tiffs, because of defectiveness.\nSecondly, i want to tell about the fact, that all students, in spite of their skills, should study together. One the one hand, i think it is good, because it is not right to divide people according their abilities, because it may give a reason for some kind of harassment. One should note here that if some student see, that all his classmates cope with a studying and he is not, he will be trying not to distinguish from his friends. \nIn conclusion, i can say that i like both of these facts, and i am concerned that people should decide by them selves if they want to study with people with the same skills level or with people with different levels.\n", "./data/exam/exam2016/OR_108_2.txt": "Nowadays it is widely discussed that children in modern families have a lack of attention and care of their parents. Both parents spend time together with kids not as much as they used to. There are several reasons which can cause this and some troubles that can appear in this situation. To begin with, career is the main obstack to spend enough time with family. In present-day world, people desire to achieve a great success, find a well-paid job and get the highest position in companies. And if they did not manage to do this earlier, before family life, they try to achieve this when they already have children. So grandmothers and grandfathers, babysitters became main people who brings up babies. In addition, lifestyle affects patterns of care and attention. In the past, children started working in very early age. They quite often go to work on fields or gardens with their family. Nowadays this can be observed in poor countries, but in developed ones parents are the only family members who have to go to job. Considering the problems that can be caused, getting into trouble can me named as the main issue. When a child, especially an adolescent, sees that parent do not care about his life he decided that it is allowed to do whatever he wants. Teenagers join bad companies and their new friends quite often have some bad habits such as drugs or alcohol. Moreover, lack of parent\u2019s attention influences on school marks of a child. Helping in learning process and participating in school life are the essential part of parenting. The way how adults bring up their children affects on their attitude to education. To sum up, help to find a proper way in further life it is significant duty of parents and it is quite important to spend enough time with children to protect them from any troubles.\n", "./data/exam/exam2014/2012-2014_2/esl_00445.txt": "A story\nSome years ago I had an old Russian car. It was ten years. As my father sad \"It was never broken down\". That means that the car never broken down before I began to use it. Each day I was founding something new that was not working: lights, accumulator, ignition and generator. Some of that list did not work and was a result of different symptoms. One day I drove on the high-speed road and began to brake. I had a long distance behind. As usually I did it slowly. Suddenly the car stopped because car brakes blocked in spite of the fact I released the brakes. It was terrible. After the car stopped in the middle of the road without any serious need, the brake pedal began to work and it was possible to move. The situation was not very good because the other car owners were not going to stop immediately. I came on the service. The \"master\" said that he knew what the problem is. After his work, nothing has changed. It took me a half of an hour to drive without any immediately stopped and after I applied the brakes the car came to a dead stop. About a week I travel from one service to another trying to find any specialist, who knows what is wrong. And I found the service. My rear brakes did not work: the drop of braking fluid was dropped on the brake shoe. After I broke several times the brake shoe dried out and the brakes blocked. I was happy that everything was ended well.\n", "./data/exam/undefined/AKhr_16_2.txt": "Some people think, that art can not be regulated by government. Other people are sure, that authorities have a right to restrict the freedom of artists to obtain stability in the society or to defend some moral rules. In this essay I will try to express my own opinion on this interesting problem. On the one hand, people have a right to express their own opinion in all situation. Freedom of speech is one of the important components of modern democratic society. The restriction of this freedom is typical for totalitarian states. Thus, in nazist Germany, many artists were arrested, because their art seemed \"inadmissible\" for Hitler's government. This way of regualion is undemocratic: I think, that all people schould have the right to critisize their own government or to create books (or pictures, or music) which is opposed to the current policy of the state. On the other hand, sometimes art can humilate. For instance, \"funny\" pictures with Mohammed's face in one of the european magazines in 2008 humiliated muslim's religious believs. Mental damage can be very serious and I think, that artists don't have a right to humiliate other people. In my opinion, the right to express your own ideas is not absolute. Every artist should understand the results of his (or her) words. In the situation of serious potential damage government can restrict the freedom of speech. In conclusion, I'd like to notice, that art is very powerful instrument to influence on people's behaviour, attitudes and mood. In difficult situations government has a right to restrict the freedom to express ideas, if this make people happier.\n", "./data/exam/exam2017/OBy_101_2.txt": "Today in our modern century there are lot\u2019s of opportunities for leisure. Music and films related many people, however, there are those who convinced that we should not pay for them and those who are believe that pirate copies and Internet downloads are illegal. The issue is rather complicated, I\u2019m going to express my personal opinion and analyse by opponent\u2019s one.\n\nFirst of all, I disagree with those sceptics, who are convinced that music and films are beloved to musicians and producers, and there should be punishment for pirate copies. In my opinion, if a person is gifted, if he had God\u2019s talent, his art (music, films, paintings, poems) is bellowed to the whole world, to everything. He should enjoy that the others make illegal copies from his art and save downloads for free. Because the art is not a business; it\u2019s a way. However, those, who are not gifted and make some arts for money also should be glad that their boring arts are useful for someone.\n\nIn spite of these arguments, there are musicians and producers and directors who lose their money from pirate and Internet copies. They investate capital and time, but today less and less people buy disks with favourite group and DVD because of Internet where is possible to download everything you want and you do not need even to go out of your flat. People who earn money by art usually want save their work not only from pirate copies but also from other people\u2019s changing, copying, cospleing. So one author want his work to laughing.\n\nMaybe there is some points in my opponent\u2019s position, but these people do not take into consideration the fact that we live in Internet century now, in the big open global world. Everything is bought and sold.\n\nTo sum up, I would like to put and emphasis on my opinion again. Copying and sharing music or films is not a crime o theft, people who download these files should not be punished. However, every opinion has its right to existence.\n", "./data/exam/exam2017/OBy_74_2.txt": "Some people believe that rise of using modern technology causes the increasing number of health problems.\n\nActually, there is a number of reasons for this issue. First  and foremost, it is clearly, that every single person living in a modern realities has his or her own smartphone. Nowadays scientists strongly believe that mobile phones can make a huge damage to human\u2019s heath-rhythm and eyesight. Secondly, logisty say that a permanent opportunity to get any kind of information you want causes many problems. For example with nerves and human\u2019s phsychology in general. Another major reason is the fact that the level of traffic in modern cities is highly increased. This causes air pollution, which is also destroying for human\u2019s health. Some researches say that nowadays ozone layer is already damaged so much, that we can\u2019t even recover it. \n\nNevertheless, there are some ways humanity could use to resolve these issues. Firstly, we can rearrange transporting use as more environmentally-friendly. For example, international transportations could make their transporting by using electric transport. Trains nowadays are as fast as cars or ships. This kind of transport is not wedespreaded because of the borders and the laws. I guess, governments should think about ecology and open borders for the international railway connection. Another thing that can be done to reduce technology damage on our health is try to face less mobile phone or a computer in your everyday life. You might use your phone only for connecting people and a compute only for work issues, for instance.\n\nTo summarise, I would like to say that influence of widespread technology on our health is not just technologies imperfectane fault. Personally, I think that it is our own choise. To ger pleasure and comfort by using our gadgets and have some problems with health or to have a great healthy body, but say \u2018no\u2019 to using technologies for your pleasure. Everything has it\u2019s worth and it is up to you to decide.\n",
              "./data/exam/exam2017/EGe_111_1.txt": "The graph depicts the usage of three diverse social networks by people aged from 18 to more than 65 in the United States. The most considerable share is represented by the group aged by 18-29 which use Facebook while the most insignificant amount is formed by retired people who are more than 65.\nIt is clear from the graph that the majority of Facebook users is formed by people aged between 18 and 29. It can be concluded that the share of users is changed corresponding with their ages. The older group is, the highest decrease in Facebook usage is observed.\nAs it can be seen, the highest amount of Instagram user is formed by the age 18-29. It is obvious that adults between the ages 30 and 49 make up the amount that is twicer less than the group that is younger than them. The most insignificant amount is represented by the group of more than 65.\nAs it be observed, approximately the same share of Linkedln is represented by the group of 30-49 and 50-64. The equal amount of users can be seen in the group of 18-29 and more than 65 the percentage of which sharply decreases in comparison to other groups.\nOverall, the most popular social networks among the youngest adult group is Facebook while the most unpopular one among them is Linkedln. The most significant share of the oldest group is observed in Linkedln usage while the most irrelevant social network among this group is Instagram.", "./data/2012-2014/esl_00950.txt": "What about me, I really like blogs. Firstly I like it as an idea, as a concept. On my opinion blogs gave as a real freedom of self-expression. Everybody can say everything to the whole word owning to Internet and world will hear him. Blogging kills a censorship, blogging equalizes us. Talented written blog can be more popular than a newspaper, so, the success of a writer or of a journalist doesn\u2019t depend of a publisher any more, now readers can decide themselves what they will read.\n\nAs far as I am concerned, I read only few blogs, if we understand by this term only personal diaries, but the news, which I prefer to read, are unofficial, in effect it is a kind of blog. I also read some personals blogs. My favorites are the blogs of the fashion journalist and of the programmer. This reading permits me to see the world in a different light, from a different perspective.\n", "./data/exam/best_works/30_1.txt": "The graph illustrates the amount of children who do not have the opportunity, to go to school in a period of time from the year 2000 to 2012 by sex and geographic location.\nThere is a general trend in all given countries including Africa and other states all over the world-girls tend to have less access to primary education than boys. So nearly 45 millions of children could not afford education in 2000 in Africa and more than half of that number is established by girls \u2013 23,7 millions to 20 millions of boys. This proportion is even more in the South Asia in the same year \u2013 21,6 millions girls compared with 11,1 millions of young men. In the rest of the world situation was the same in 2000 \u2013 12,8 millions girls to 10,5 millions boys. The trend began to ruin by 2012. All selected countries changed the politics to gender factor. So, in South Asia and the rest of the world the proportion between uneducated boys and girls became almost the same at that time \u2013 4,8 millions of girls to 5,1 millions boys and 7,6 millions of girls to 7,7 millions of boys. The only exception is Africa \u2013 the number of girls who don\u2019t go to school because less but as in the past, boys tend to have more access to the educational programs.\nAll in all, we can see that the amount of uneducated children falls down each year and girls now have nearly the same opportunities for studying as boys do.\n", "./data/exam/exam2017_6/NMya_25_2.txt": "The crime activity among younger generation is the issue of concern in many countries. The reasons of increased attention to this topic is the jump of the frequency of law breaks among youngers. What is the cause of such unusual growing of criminal activity? And what steps should society take to prevent futher increasing of crime commiting children and teens? As for my opinion, the main reason to law disobediance is lack of support from other parts of society. That is why parents and teachers should be ones to fix this situation by giving more attention to youngers they look after.\nThe social pressure, stress, depression, troubles with friends, first love and lack of time are the main problems youngest generation faces very frequently and knows too good. Teens hormonal changes and mental distability prevents them from solving these problems successfully all alone. That is why they tend to solve situations in a very radical and dramatical way, which often leads to commiting a crime. On the other hand, almost all young people have or should have special grown ups to look after them and help to become a good part of society. Lack of this help leads to the stated problem of growing criminality among youngs.\nThus, parents and teachers as people closest to young generation should take the situation under control. The most obvious idea to do it is to give more attention to children to children and teenagers. Indeed, it is known, that it is common situation, that parents are too busy earning money to get into all details of their child life As for teachers, they often have too much people in class to give proper attention to each individual. But from the other side it is possible for both parents and teachers to cooperate and notice worrying tendentions together. For example, teacher is able to feel changes in whole behavior pattern of his students and give know parents in case something wrong, and then they would take a closer look on changes in their child behaviour.\nAll in all, it is clear that rising crime committment rate among youngsters is a fact society cannot close eyes on. That is why both parents and teachers should give much attention to younger generation in order to prevent this situation to continue by giving younger people support they need.\n", "./data/old IELTS/IELTS2015/MTsy_28_2.txt": "Nowadays many people believe that health problems in many countries can be solved by opportunity to do sports that should be given to any individual. On the other side, we have the ones that arguing: they consider that sports id not the only necessary measure should be taken.\nUndoubtedly, there's many arguments can be given for increasing sport facilities and vice versa: let's take a look at the most important ones. First of all, it's the kids we can speak about: in the period their small curious minds are being formed, it seems to be pretty easy to take the wrong path. I've seen by myself how families are been literally ruined by the kid's drug usage: in many ways it can be prevented by government giving various sports doing opportunities: sometimes children just don't know what to do with their time. Furthermore, there's one more interesting tendention that takes place: poor families are most tending to use. So we can conclude if they'll not have to pay to do some certain sports, they may choose the sportsman's way.\nThese arguments are seem to be pretty ensurable, aren't they? Nevertheless, we have disagreed individuals having their own opinion. First thing need to be said against such kind of policy is that sport is most essential and natural kind of activity we can even think of! All one has to do to start jogging is to buy a couple of sneakers - it's that easy! What about the profit? It seems to be way more attractive than living not longer than 40 years because of a heart attack or a stroke. The main thesis here is: \"People already have big opportunities. They choose their own way by themselves\". Moreover, there's a lot more measures needed to be taken by government: for instance, some people say that free professional education can solve a lot more concerns more effectively.\nI strongly believe that healthcare of every single person is his own responsibility. More than that, I think that such kind of responsibility goes from inside, via our thoughts and beliefs. So here comes the conclusion: I support second point of view and think that to start jogging you need not the buildings, but the legs.\n", "./data/old IELTS/IELTS2016/JSl_67_2.txt": "Nowadays it is more valuable for international companies move their factories and other types of business making to developing countries and employ local workers because prime cost of their labour is lower than in developed countries. It is a control ersial issue but I personally do not agree with the statement that the advantages of such kind of situation can outweigh the disadvantages. In my opinion moving business to developing countries may cause several problems in both of developed and developing ones. Firstly, from the last decade it can be seen that for example European countries not much work places for all workers that need them. A big amount of unemployment people which are more likely to do crimes creating other unpredictable problems may also cause distabilisstion in political area. It I always not good for both politicians, citizens and other types of people. Secondly, if many business move their businesses to poorer countries, developing of this countries may not continue as fast as it was in the past. Low cost of labour provides good conditions for foreign businessmen not for the native ones. Unequal will remain the same trend and developing countries may never become developed. If the country work not for itself but for the other country it just stops its developing. In conclusion I want to say that moving business and employing workers in developing countries have a big number of disadvantages and if we will think not only about nearest future we will understand that it is bad for people who live in both types of countries when companies make unequally between developing and developed countries at the situation like this. And moving businesses cause unequally in 90% of all amount of this kinds of situations.  \n",
                            "./data/old IELTS/IELTS2016/EKu_72_2.txt": "One of the highly argued topics today is whether poorer countries are better for the international companies to locate there their business there or not. This essay is going to cover some advantages and disadvantages of this idea. One of the greatest advantages is the cost of labor in poor and developing countries. It is rather difficult to find proper job in such countries and people there are usually not as highly educated as people in developed ones. Also the standards and prices of living in poorer countries are lower. So, having all this concerned international companies do not have to pay too much for the work done by these people. This strategy can also broaden the market of this company as people living in the county where the product is being produced would likely buy it. As for disadvantages, lower level of education means lower quality of the work done. If people have no knowledge of how to do some kind of work, they need quite a lot of time to learn how to do it and still it does not prove that they would learn how to do it properly. And as lower quality of the product means losses in sales it can be dramatic for the company to locate their business in a developing country. Also, locating their production in another country international companies risk the secrets of their product to be mastered or sold out by a factory in that developing country. So, trying to benefit from such strategy company may ruin completely.  \u00a0 All in all, general statistics show that many greatest figures on an international market have not yet suffered from locating their producing factories in developing but not the poorest countries with great amount of population.", "./data/2012-2014/esl_00837.txt": "The table shows how many students of both sexes, from different territories enrolled at Bristol University in three periods of time: in 1928, 1958 and 2008.\nIt is clear from the table that its the most significant feature is that in 2008 6377 students were registered, also in this year there was a first time than quantity of male students was less than number of females. We can see that in 1958 there were 1046 enrolled students 68 per cent of them were males. In 1928 the lowest number of students were fixed, it was about 218 people, however in 50 percent of these undergraduates came from within 30 miles of Bristol, which became the highest gauge in this area. A more detailed at the table reveals, that in 2008 28 per cent of students came from 98 different countries, which means that Bristol University appear at an international level.\nTo sum up, the table provided us with information, which convinced us that education in universities becomes more accessible for everyone. \n"}

outie = [print_annotated_webpage(annotate(test_texts[tt]), tt) for tt in test_texts]
with open(curname, 'w', encoding="utf-8") as w:
  json.dump(outie, w)

INFO:tensorflow:Writing example 0 of 128


I0430 22:14:40.701822 139855783298816 tf_logging.py:115] Writing example 0 of 128


INFO:tensorflow:*** Example ***


I0430 22:14:40.705250 139855783298816 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0430 22:16:30.343667 139855783298816 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0430 22:16:30.424796 139855783298816 tf_logging.py:115] Done running local_init_op.




Annotation took time  0:00:08.080651


In [94]:
os.listdir(".")

['.ipython',
 '.cache',
 '.ipynb_checkpoints',
 'delete.json',
 '.jupyter',
 '.bashrc-anaconda3.bak',
 '.config',
 'anaconda3',
 'Anaconda3-4.0.0-Linux-x86_64.sh',
 'download_google_drive',
 '.nano',
 '.bash_history',
 'gdrive-linux-x64',
 '.keras',
 'delete_test.json',
 '.python_history',
 'nltk_data',
 'wsites_delete_ToFifteen.json',
 'Datasets',
 '.local',
 'wsites_lex_item_choice_AugmentedRatio.json',
 'jupyter-drive',
 'cuda-repo-ubuntu1604_9.0.176-1_amd64.deb',
 '.continuum',
 '.conda',
 '.bashrc',
 '.nv',
 'bert_output',
 'cuda-repo-ubuntu1604_8.0.61-1_amd64.deb',
 '.gnupg',
 'REALEC_AutoAnnotator_Generic_BERT_pipeline_Instance_2-Copy1.ipynb',
 '.gdrive',
 'lex_item_choice_test.json',
 '.profile',
 '.wget-hsts',
 'REALEC_AutoAnnotator_Generic_BERT_pipeline_Instance_2.ipynb',
 '.bash_logout',
 '.ssh',
 'lex_item_choice.json']

In [47]:
outie[3]

'<html>\n<head>\n<title>./data/exam/exam2017/VSa_75_1.txt</title>\n<meta charset="utf-8">\n<style type="text/css">\n.red {\n\tbackground: #ffdddd;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>The graph provide <div class="red">us</div> <div class="red">information</div> about print and eBook market in USA, Germany, and UK in 2014 annd 2018. The biggest number in this tip of print and eBook in USA. HowEver, number of eBook and print market also big in Germany. In China we can see less number of market and in the UK this number the smallest between all country in 2014 and 2018. In USA\'s graph we see that number of eBook market increas in 2018 on 3,5% comparasion with 2014 <div class="red">and</div> number of print market desceased on 3% in 2018. In Germany number of print market stay the same in 2018 <div class="red">as</div> in 2014 but number of eBook market small increas in 2018. This number is 0,5%. In China number of print fall in 2018 on 0,6% but number of E book increas

In [56]:
!./gdrive-linux-x64 upload wsites_delete_ToFifteen.json

Uploading wsites_delete_ToFifteen.json
Uploaded 1pdhf87lAeZ9mWfx7X_mfZtd71QzWaMpL at 27.5 KB/s, total 27.5 KB


In [33]:
!ls -lAh ./bert_delete

total 7.4G
-rw-rw-r-- 1 ivantorubarov ivantorubarov  277 Apr 30 20:15 checkpoint
drwxr-xr-x 2 ivantorubarov ivantorubarov 4.0K Apr 30 21:40 eval
-rw-rw-r-- 1 ivantorubarov ivantorubarov 409M Apr 30 20:15 events.out.tfevents.1556650540.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov 391M Apr 30 18:56 graph.pbtxt
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G Apr 30 20:03 model.ckpt-10500.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K Apr 30 20:03 model.ckpt-10500.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov 103M Apr 30 20:03 model.ckpt-10500.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G Apr 30 20:06 model.ckpt-11000.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K Apr 30 20:06 model.ckpt-11000.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov 103M Apr 30 20:06 model.ckpt-11000.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G Apr 30 20:09 model.ckpt-11500.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K Apr 30 20:09 model.ckpt

In [97]:
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-12378.data-00000-of-00001
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-12378.index
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-12378.meta

Uploading ./bert_output/model.ckpt-12378.data-00000-of-00001
Uploaded 1RAYvy_FPUzbrlC4M-Q_33FQ3ULBrE003 at 39.0 MB/s, total 1.3 GB
Uploading ./bert_output/model.ckpt-12378.index
Uploaded 1ZS1_qHp28iMXKVJi7FQbKrvSDBjqnDzK at 23.5 KB/s, total 23.5 KB
Uploading ./bert_output/model.ckpt-12378.meta
Uploaded 1mW1wa7ZNGwAeuWaGLRJMhApXR7nBnGNP at 33.1 MB/s, total 107.6 MB


In [55]:
!rm -rf ./bert_output

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': curname})
uploaded.SetContentFile('./'+curname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 16_pmUKOHS5lKXM2o-1QyVcPQohalCRZd


# Preparing existing model for REALEC production